In [1]:
### BLOCK 1/8 — Imports & Setup
# ==========================================================
# Artwork Analyzer v17 — Normal Visuals + Headed Summaries
# ==========================================================
# • Local-only (OpenCV + NumPy + Pillow)
# • Dynamic scaling, safe gamma
# • Separate overlay outputs per analysis type
# • Encouraging curatorial tone
# ==========================================================

import os, json, time, textwrap, math
import numpy as np
import cv2
from PIL import Image
from math import atan2, degrees

# Fixed width for analysis panel (right side of each output)
PANEL_W = 420

# -----------------------------
# Utility Functions
# -----------------------------
def ensure_dir(p): os.makedirs(p, exist_ok=True)

def load_rgb(path):
    """Load an image as RGB numpy array."""
    img = Image.open(path).convert("RGB")
    return np.array(img)

def to_uint8(img_f32):
    """Convert [0–1] float array to uint8 safely."""
    return np.clip(img_f32 * 255.0, 0, 255).astype(np.uint8)

def safe_gamma(img_u8):
    """Adjust gamma gently to balance brightness for display."""
    img = img_u8.astype(np.float32) / 255.0
    m = float(img.mean())
    if m < 0.25:       # dark images
        img = np.power(img, 0.7)
    elif m > 0.80:     # bright images
        img = np.power(img, 1.1)
    return to_uint8(np.clip(img, 0, 1))

def resize_min_pixels(img, target_pixels=1_600_000):
    """Upscale small images to improve analysis fidelity."""
    H, W = img.shape[:2]
    cur = H * W
    if cur >= target_pixels:
        return img, 1.0
    scale = math.sqrt(target_pixels / float(cur))
    newW, newH = int(W * scale), int(H * scale)
    new = cv2.resize(img, (newW, newH), interpolation=cv2.INTER_CUBIC)
    return new, scale

def wrap_text_lines(text, width_chars=80):
    """Wrap long text neatly for display in panel."""
    return textwrap.wrap(text, width=width_chars)

def json_safe(obj):
    """Recursively convert numpy objects to JSON-safe types."""
    if isinstance(obj, dict):
        return {k: json_safe(v) for k, v in obj.items()}
    elif isinstance(obj, (list, tuple)):
        return [json_safe(x) for x in obj]
    elif isinstance(obj, (np.generic, np.number)):
        return obj.item()
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj

def norm01(x, lo, hi):
    """Normalize x into [0,1] given lower and upper bounds."""
    if hi <= lo: return 0.0
    return float(np.clip((x - lo) / (hi - lo), 0, 1))
### BLOCK 2/8 — Color & Palette Analysis
# ==========================================================

def ciede2000(Lab1, Lab2):
    """Compute CIEDE2000 color difference between two LAB triplets."""
    L1,a1,b1 = Lab1; L2,a2,b2 = Lab2
    C1 = np.sqrt(a1*a1 + b1*b1); C2 = np.sqrt(a2*a2 + b2*b2)
    avg_C = (C1 + C2)/2.0
    G = 0.5*(1 - np.sqrt((avg_C**7)/((avg_C**7)+25**7)))
    a1p, a2p = (1+G)*a1, (1+G)*a2
    C1p, C2p = np.sqrt(a1p*a1p + b1*b1), np.sqrt(a2p*a2p + b2*b2)
    h1p = (np.degrees(np.arctan2(b1,a1p))+360)%360
    h2p = (np.degrees(np.arctan2(b2,a2p))+360)%360
    dLp = L2-L1; dCp = C2p-C1p
    dhp = h2p-h1p
    if C1p*C2p==0: dhp=0
    elif dhp>180: dhp-=360
    elif dhp<-180: dhp+=360
    dHp = 2*np.sqrt(C1p*C2p)*np.sin(np.radians(dhp)/2.0)
    avg_Lp=(L1+L2)/2.0
    if C1p*C2p==0: avg_hp=h1p+h2p
    elif abs(h1p-h2p)>180:
        avg_hp=(h1p+h2p+360)/2.0 if (h1p+h2p)<360 else (h1p+h2p-360)/2.0
    else:
        avg_hp=(h1p+h2p)/2.0
    T=(1-0.17*np.cos(np.radians(avg_hp-30))
        +0.24*np.cos(np.radians(2*avg_hp))
        +0.32*np.cos(np.radians(3*avg_hp+6))
        -0.20*np.cos(np.radians(4*avg_hp-63)))
    d_ro=30*np.exp(-((avg_hp-275)/25)**2)
    avg_Cp=(C1p+C2p)/2.0
    Rc=2*np.sqrt(((avg_Cp**7)/((avg_Cp**7)+25**7)))
    Sl=1+0.015*(avg_Lp-50)**2/np.sqrt(20+(avg_Lp-50)**2)
    Sc=1+0.045*avg_Cp
    Sh=1+0.015*avg_Cp*T
    Rt=-np.sin(np.radians(2*d_ro))*Rc
    return float(np.sqrt((dLp/Sl)**2 + (dCp/Sc)**2 + (dHp/Sh)**2 + Rt*(dCp/Sc)*(dHp/Sh)))

def _kmeans_lab(lab_img, k, attempts=10):
    """Run K-means on LAB pixels to find k color clusters."""
    H,W=lab_img.shape[:2]
    Z=lab_img.reshape(-1,3).astype(np.float32)
    crit=(cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER,80,0.2)
    _,labels,centers=cv2.kmeans(Z,k,None,crit,attempts,cv2.KMEANS_PP_CENTERS)
    return labels.reshape(H,W), centers.astype(np.float32)

def _palette_quality(centers, entropy):
    """Assess palette diversity using ΔE distances + entropy."""
    if centers.shape[0]<2: return entropy,0.0
    dEs=[ciede2000(centers[i],centers[j])
         for i in range(len(centers)) for j in range(i+1,len(centers))]
    mean_dE=float(np.mean(dEs)) if dEs else 0.0
    spread=float(np.std(centers,axis=0).mean())
    return 0.6*(0.7*mean_dE+0.3*spread)+0.4*entropy, mean_dE

def color_analysis(img, ks=(6,8,10)):
    """Extract palette, compute harmony, ΔE, and hue histogram."""
    H,W=img.shape[:2]
    lab=cv2.cvtColor(img,cv2.COLOR_RGB2LAB)
    hsv=cv2.cvtColor(img,cv2.COLOR_RGB2HSV)
    hue=(hsv[:,:,0].astype(np.float32)*2.0)%360.0
    hue_hist,_=np.histogram(hue,bins=36,range=(0,360))
    hue_hist=hue_hist.astype(np.float32)

    best=None; best_score=-1
    for k in ks:
        labels,centers=_kmeans_lab(lab,k)
        counts=np.bincount(labels.reshape(-1),minlength=k).astype(np.float32)
        frac=counts/(H*W+1e-6)
        p=frac[frac>0]
        entropy=-float(np.sum(p*np.log2(p)))
        score,mean_dE=_palette_quality(centers,entropy)
        if score>best_score: best_score=score; best=(k,labels,centers,counts,entropy,mean_dE)

    k_sel,labels,centers,counts,entropy,mean_dE=best
    rgb_centers=cv2.cvtColor(centers.reshape(-1,1,3).astype(np.uint8),
                              cv2.COLOR_Lab2RGB).reshape(-1,3)
    palette=[tuple(map(int,c)) for c in rgb_centers]

    a,b=centers[:,1]-128, centers[:,2]-128
    hues_c=(np.degrees(np.arctan2(b,a))+360)%360
    warm_ratio=float(((hues_c<90)|(hues_c>330)).sum()/len(hues_c))
    cool_ratio=float(((hues_c>180)&(hues_c<300)).sum()/len(hues_c))
    if np.ptp(hues_c)<35: harmony="analogous"
    elif any(abs(hues_c[i]-hues_c[j])%360>150 for i in range(len(hues_c)) for j in range(i+1,len(hues_c))):
        harmony="complementary"
    else: harmony="mixed"

    # ΔE proxy map via Lab gradients
    labf=lab.astype(np.float32)
    gxL=cv2.Sobel(labf[:,:,0],cv2.CV_32F,1,0); gyL=cv2.Sobel(labf[:,:,0],cv2.CV_32F,0,1)
    gxa=cv2.Sobel(labf[:,:,1],cv2.CV_32F,1,0); gya=cv2.Sobel(labf[:,:,1],cv2.CV_32F,0,1)
    gxb=cv2.Sobel(labf[:,:,2],cv2.CV_32F,1,0); gyb=cv2.Sobel(labf[:,:,2],cv2.CV_32F,0,1)
    grad_mag=np.sqrt((gxL**2+gyL**2)+(gxa**2+gya**2)+(gxb**2+gyb**2))
    contrast_map=(grad_mag-grad_mag.min())/(grad_mag.max()-grad_mag.min()+1e-6)

    return dict(
        palette_rgb=palette,
        palette_entropy=float(entropy),
        mean_deltaE=float(mean_dE),
        warm_ratio=warm_ratio,
        cool_ratio=cool_ratio,
        harmony=harmony,
        contrast_map=contrast_map,
        hue_hist=hue_hist,
        k_selected=int(k_sel)
    )
### BLOCK 3/8 — Texture Analysis (Gabor Pyramid + Roughness)
# ==========================================================

def gabor_pyramid(gray, scales=(3,5,7,9,11,13), orientations=12):
    """Analyze texture energy and directionality using multi-scale Gabor filters."""
    H, W = gray.shape
    angles = np.linspace(0, 180, orientations, endpoint=False)
    combined = np.zeros((H, W), np.float32)
    angle_energy = np.zeros(orientations, np.float32)

    for oi, theta in enumerate(angles):
        theta_rad = np.deg2rad(theta)
        e_sum = np.zeros((H, W), np.float32)
        for k in scales:
            kernel = cv2.getGaborKernel((21,21), sigma=k, theta=theta_rad,
                                        lambd=10, gamma=0.5, psi=0)
            resp = cv2.filter2D(gray, cv2.CV_32F, kernel)
            e_sum += resp * resp
        combined = np.maximum(combined, e_sum)
        angle_energy[oi] = e_sum.mean()

    anisotropy = float((angle_energy.max() - angle_energy.min()) /
                       (angle_energy.mean() + 1e-6))
    dom_angle = float(angles[int(np.argmax(angle_energy))])
    gmap = (combined - combined.min()) / (combined.max() - combined.min() + 1e-6)

    # Texture entropy and roughness
    hist, _ = np.histogram(gray, bins=256, range=(0,256))
    hist = hist.astype(np.float32)
    hist /= (hist.sum() + 1e-6)
    entropy = -float(np.sum(hist * np.log2(hist + 1e-12)))
    roughness = float(gray.std())

    return dict(
        gabor_angles=angles,
        gabor_energy=angle_energy,
        gabor_map=gmap,
        gabor_anisotropy=anisotropy,
        gabor_dom_angle=dom_angle,
        lbp_entropy=entropy,
        lbp_roughness=roughness
    )

# -----------------------------
# Texture / Contrast / Saliency Strength Helpers
# -----------------------------
def texture_strength(T):
    """Blend anisotropy + roughness into [0–1] texture visibility weight."""
    a = norm01(T["gabor_anisotropy"], 0.15, 0.85)
    r = norm01(T["lbp_roughness"], 8.0, 35.0)
    return 0.6 * a + 0.4 * r

def contrast_strength(C):
    """Map mean ΔE to visibility scale."""
    return norm01(C["mean_deltaE"], 6.0, 28.0)

def saliency_strength(mask):
    """Estimate visibility strength from normalized subject area."""
    area = float(mask.sum()) / 255.0
    return norm01(area, 0.04 * mask.size, 0.45 * mask.size)
### BLOCK 4/8 — Saliency & Subject Detection
# ==========================================================

def spectral_saliency(grayf, blur=(9,9), sigma=2):
    """Compute spectral residual saliency map."""
    fft = np.fft.fft2(grayf)
    loga = np.log(np.abs(fft) + 1e-8)
    spec = np.exp((loga - cv2.blur(loga, (3,3))) + 1j * np.angle(fft))
    sal = np.abs(np.fft.ifft2(spec))**2
    sal = cv2.GaussianBlur(sal, blur, sigma)
    sal = (sal - sal.min()) / (sal.max() - sal.min() + 1e-6)
    return (sal * 255).astype(np.uint8)

def subject_from_saliency_fused(s1, s2):
    """Fuse two saliency maps and return subject mask + bounding box."""
    H, W = s1.shape
    m1 = cv2.threshold(s1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    m2 = cv2.threshold(s2, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    m = cv2.bitwise_and(m1, m2)
    m = cv2.morphologyEx(m, cv2.MORPH_OPEN, np.ones((5,5), np.uint8))
    m = cv2.morphologyEx(m, cv2.MORPH_CLOSE, np.ones((7,7), np.uint8))

    cnts,_ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if cnts:
        x, y, w, h = cv2.boundingRect(max(cnts, key=cv2.contourArea))
    else:
        x, y, w, h = W//4, H//4, W//2, H//2
        cv2.rectangle(m, (x,y), (x+w,y+h), 255, -1)
    return dict(mask=m, bbox=(x,y,w,h))

def saliency_analysis(img):
    """Run dual-scale saliency and derive focal point and subject mask."""
    g = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY).astype(np.float32)
    s_coarse = spectral_saliency(cv2.GaussianBlur(g, (0,0), 3.0), blur=(11,11), sigma=3)
    s_fine   = spectral_saliency(cv2.GaussianBlur(g, (0,0), 1.0), blur=(7,7), sigma=1.5)
    fused = np.maximum(s_coarse, s_fine)
    sub = subject_from_saliency_fused(s_coarse, s_fine)

    Y, X = np.indices(fused.shape)
    w = fused.astype(np.float32) + 1
    cx, cy = int((X * w).sum() / w.sum()), int((Y * w).sum() / w.sum())

    return dict(
        saliency_map=fused,
        focal=(cx, cy),
        subject_mask=sub["mask"],
        subject_bbox=sub["bbox"]
    )
### BLOCK 5/8 — Composition & Structure Analysis
# ==========================================================

def _line_intersection(p1, p2):
    """Find intersection between two lines, or None if parallel."""
    (x1,y1,x2,y2),(x3,y3,x4,y4)=p1,p2
    den=(x1-x2)*(y3-y4)-(y1-y2)*(x3-x4)
    if abs(den)<1e-6: return None
    px=((x1*y2-y1*x2)*(x3-x4)-(x1-x2)*(x3*y4-y3*x4))/den
    py=((x1*y2-y1*x2)*(y3-y4)-(y1-y2)*(x3*y4-y3*x4))/den
    return (px,py)

def _cluster_points(points, grid):
    """Cluster points into grid cells to find dominant intersection region."""
    if not points: return None
    pts=np.array(points,dtype=np.float32)
    xs,ys=pts[:,0],pts[:,1]
    xbin=np.round(xs/grid).astype(int)
    ybin=np.round(ys/grid).astype(int)
    keys=xbin*100000+ybin
    uniq,counts=np.unique(keys,return_counts=True)
    best_key=uniq[np.argmax(counts)]
    mask=(keys==best_key)
    bx=float(xs[mask].mean()); by=float(ys[mask].mean())
    return (bx,by)

def composition_analysis(img, focal_xy):
    """Detect lines, compute vanishing point, thirds/golden ratios, and symmetry."""
    g=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    H,W=g.shape
    edges=cv2.Canny(g,80,200)

    # Multi-pass Hough line detection
    lines=[]
    params=[
        (1,np.pi/180,100,min(H,W)//12,15),
        (1,np.pi/180,80,min(H,W)//14,20),
        (1,np.pi/180,120,min(H,W)//10,15)
    ]
    for rho,theta,thr,minL,gap in params:
        linesP=cv2.HoughLinesP(edges,rho,theta,thr,
                               minLineLength=minL,maxLineGap=gap)
        if linesP is not None:
            for l in linesP:
                ln=tuple(map(int,l[0]))
                if math.hypot(ln[2]-ln[0],ln[3]-ln[1])>=min(H,W)//14:
                    lines.append(ln)
    lines=lines[:300]

    # Vanishing point estimation
    inters=[]
    for i in range(len(lines)):
        for j in range(i+1,len(lines)):
            ip=_line_intersection(lines[i],lines[j])
            if ip is None: continue
            if -W*0.5<=ip[0]<=W*1.5 and -H*0.5<=ip[1]<=H*1.5:
                inters.append(ip)
    vp=_cluster_points(inters,grid=max(20,min(H,W)//30))
    if vp is None: vp=(W/2,H/2)

    # Focal alignment vs thirds / golden ratio
    fx,fy=focal_xy
    thirds_pts=[(W/3,H/3),(2*W/3,H/3),(W/3,2*H/3),(2*W/3,2*H/3)]
    gfac=0.618
    golden_pts=[(gfac*W,gfac*H),((1-gfac)*W,gfac*H),
                (gfac*W,(1-gfac)*H),((1-gfac)*W,(1-gfac)*H)]
    diag=np.hypot(W,H)
    thirds_delta=min(np.hypot(fx-x,fy-y) for (x,y) in thirds_pts)/(diag+1e-6)
    golden_delta=min(np.hypot(fx-x,fy-y) for (x,y) in golden_pts)/(diag+1e-6)

    # Symmetry and edge balance
    g_f=g.astype(np.float32)/255.0
    left=g_f[:,:W//2]; right=np.fliplr(g_f[:,W-left.shape[1]:])
    mse=float(np.mean((left-right)**2))
    symmetry=float(1.0 - min(1.0, mse*4.0))
    lw,rw=float(np.sum(edges[:,:W//2])),float(np.sum(edges[:,W//2:]))
    balance=(rw-lw)/(lw+rw+1e-6)

    return dict(
        lines=lines,
        vanishing_point=vp,
        thirds_delta=thirds_delta,
        golden_delta=golden_delta,
        symmetry=symmetry,
        edge_balance=balance
    )

def line_strength(lines):
    """More lines = stronger composition indicator."""
    return norm01(len(lines), 20, 260)
### BLOCK 6/8 — Illumination & Light Direction
# ==========================================================

def illumination_analysis(img):
    """Analyze light direction and tonal skew in LAB space."""
    L = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)[:,:,0].astype(np.float32)
    mu, sig = L.mean(), L.std() + 1e-6
    skew = float((((L - mu) / sig) ** 3).mean())

    # Compute directional gradients on brightest 10%
    gx, gy = cv2.Sobel(L, cv2.CV_32F, 1, 0), cv2.Sobel(L, cv2.CV_32F, 0, 1)
    hi = (L > np.percentile(L, 90)).astype(np.uint8)

    if hi.sum() > 0:
        gx_sum, gy_sum = (gx * hi).sum(), (gy * hi).sum()
        ang = float((degrees(atan2(-gy_sum, -gx_sum)) + 360) % 360)
    else:
        ang = None

    return dict(
        lightness_skew=skew,
        light_direction_deg=ang
    )

def light_strength(I):
    """Map illumination signal strength to overlay visibility."""
    if I["light_direction_deg"] is None:
        return 0.0
    s = abs(I["lightness_skew"])
    return max(0.15, norm01(s, 0.1, 0.8))
### BLOCK 7/8 — Overlays & Visual Rendering (Clear, Dynamic Indicators)
# ==========================================================
# -----------------------------
# ROI Interest Map (dynamic fusion)
# -----------------------------
def _norm01(x):
    x = x.astype(np.float32)
    mn, mx = float(x.min()), float(x.max())
    if mx - mn < 1e-6: 
        return np.zeros_like(x, np.float32)
    return (x - mn) / (mx - mn)

def _gaussian_from_point(H, W, cx, cy, sigma_frac=0.25):
    """Return a 0..1 gaussian bump centered at (cx,cy) with sigma as fraction of diag."""
    yy, xx = np.indices((H, W))
    d2 = (xx - cx)**2 + (yy - cy)**2
    diag = np.hypot(W, H)
    sigma = (sigma_frac * diag)
    g = np.exp(-d2 / (2.0 * (sigma**2) + 1e-6))
    return g.astype(np.float32)

def _line_density_map(H, W, lines, blur_sigma=2.0):
    """Rasterize lines into a soft density map (0..1)."""
    m = np.zeros((H, W), np.uint8)
    if lines:
        for (x1, y1, x2, y2) in lines:
            cv2.line(m, (x1, y1), (x2, y2), 255, 1, cv2.LINE_AA)
    m = cv2.GaussianBlur(m, (0, 0), blur_sigma)
    return _norm01(m)

def roi_interest_map(C, T, S, P, I, shape):
    """
    Build a single interest map 0..1 from:
      - saliency
      - texture energy (Gabor)
      - ΔE proxy (Lab gradient)
      - proximity to focal + vanishing point
      - line density
    Weights adapt mildly to global complexity.
    """
    H, W = shape

    # base maps (ensure correct size + normalized)
    # saliency
    sal = S["saliency_map"].astype(np.float32)
    if sal.shape[:2] != (H, W):
        sal = cv2.resize(sal, (W, H), interpolation=cv2.INTER_CUBIC)
    sal = _norm01(sal)

    # texture (Gabor)
    gmap = T["gabor_map"].astype(np.float32)
    if gmap.shape[:2] != (H, W):
        gmap = cv2.resize(gmap, (W, H), interpolation=cv2.INTER_CUBIC)
    gmap = cv2.GaussianBlur(gmap, (0, 0), 1.0)
    gmap = _norm01(gmap)

    # ΔE proxy (Lab gradient)
    dE = C["contrast_map"].astype(np.float32)
    if dE.shape[:2] != (H, W):
        dE = cv2.resize(dE, (W, H), interpolation=cv2.INTER_CUBIC)
    dE = cv2.GaussianBlur(dE, (0, 0), 1.0)
    dE = _norm01(dE)

    # proximity fields (gaussian bumps)
    fx, fy = S["focal"]
    prox_focal = _gaussian_from_point(H, W, fx, fy, sigma_frac=0.22)
    vx, vy = P.get("vanishing_point", (W/2, H/2))
    prox_vp = _gaussian_from_point(H, W, vx, vy, sigma_frac=0.28)

    # line density
    line_map = _line_density_map(H, W, P["lines"], blur_sigma=2.0)

    # measure complexity → softly modulate weights
    n_lines = len(P["lines"])
    comp_complex = np.tanh(n_lines / 120.0)               # 0..~1
    tex_complex  = np.tanh(float(T["lbp_roughness"]) / 30.0)
    con_complex  = np.tanh(float(C["mean_deltaE"]) / 20.0)

    # base weights
    w_sal = 0.34
    w_tex = 0.22
    w_de  = 0.22
    w_foc = 0.14
    w_lin = 0.08

    # adapt weights slightly to content
    w_tex *= (0.9 + 0.3*tex_complex)
    w_de  *= (0.9 + 0.3*con_complex)
    w_lin *= (0.9 + 0.3*comp_complex)

    # normalize to sum 1.0
    w_sum = (w_sal + w_tex + w_de + w_foc + w_lin)
    w_sal /= w_sum; w_tex /= w_sum; w_de /= w_sum; w_foc /= w_sum; w_lin /= w_sum

    # fuse
    fused = (w_sal*sal +
             w_tex*gmap +
             w_de *dE  +
             w_foc*0.6*prox_focal + w_foc*0.4*prox_vp +
             w_lin*line_map)

    # smooth and normalize
    fused = cv2.GaussianBlur(fused, (0, 0), 1.2)
    fused = _norm01(fused)
    return fused

def roi_interest_heatmap_overlay(img_rgb, interest_f01, alpha=0.35):
    """Debug: overlay a color heatmap of interest on the RGB image."""
    H, W = interest_f01.shape[:2]
    heat_u8 = (np.clip(interest_f01, 0, 1) * 255).astype(np.uint8)
    heat_bgr = cv2.applyColorMap(heat_u8, cv2.COLORMAP_PLASMA)
    heat_rgb = cv2.cvtColor(heat_bgr, cv2.COLOR_BGR2RGB)
    base = img_rgb.astype(np.float32)
    overlay = (1 - alpha) * base + alpha * heat_rgb.astype(np.float32)
    return overlay.astype(np.uint8)

def overlays_on_artwork(img, C, T, S, P, I, mode="all"):
    """Render dynamic color-coded overlays onto the artwork."""
    base = img.astype(np.float32) / 255.0
    H, W = base.shape[:2]

    # Strength scaling for transparency
    w_tex   = texture_strength(T)
    w_de    = contrast_strength(C)
    w_sal   = saliency_strength(S["subject_mask"])
    w_lines = line_strength(P["lines"])
    w_light = light_strength(I)

    # 1. Texture (amber haze)
    gmap = T["gabor_map"]
    gmap = cv2.GaussianBlur(gmap, (0,0), 1.0)
    gmap = (gmap - gmap.min()) / (gmap.max() - gmap.min() + 1e-6)
    tex_layer = np.dstack([gmap, gmap * 0.45, 0 * gmap]) * (0.35 * w_tex)

    # 2. Local color contrast (magenta haze)
    contrast = C["contrast_map"]
    contrast = cv2.GaussianBlur(contrast, (0,0), 1.0)
    contrast = (contrast - contrast.min()) / (contrast.max() - contrast.min() + 1e-6)
    de_layer = np.dstack([contrast, 0 * contrast, contrast]) * (0.28 * w_de)

    # 3. Saliency (soft red haze)
    sal = S["saliency_map"].astype(np.float32)
    if sal.shape[:2] != (H, W):
        sal = cv2.resize(sal, (W, H), interpolation=cv2.INTER_CUBIC)
    sal = cv2.normalize(sal, None, 0, 1, cv2.NORM_MINMAX)
    sal_layer = np.dstack([sal, sal * 0.45, sal * 0.45]) * (0.22 * w_sal)

    # 4. Composition lines (cyan/green/yellow)
    line_layer = np.zeros_like(base)
    for (x1, y1, x2, y2) in P["lines"][:240]:
        ang = (np.rad2deg(np.arctan2(y2 - y1, x2 - x1)) + 180) % 180
        col = (0, 1, 0) if 60 < ang < 120 else (0, 0, 1) if ang < 30 or ang > 150 else (1, 1, 0)
        cv2.line(line_layer, (x1, y1), (x2, y2), (col[2], col[1], col[0]), 1, cv2.LINE_AA)
    line_layer = cv2.GaussianBlur(line_layer, (0,0), 2) * (0.45 * w_lines)
    # Optional single-mode rendering
    if mode == "color":
        comp = base * 0.9 + de_layer
    elif mode == "texture":
        comp = base * 0.9 + tex_layer
    elif mode == "saliency":
        comp = base * 0.9 + sal_layer
    elif mode == "composition":
        comp = base * 0.9 + line_layer
    elif mode == "illumination":
        comp = base.copy()
        # Initialize comp_bgr before drawing
        comp_bgr = (comp[:, :, ::-1] * 255).astype(np.uint8).copy()
        if I["light_direction_deg"] is not None:
            cx, cy = W - 60, 60
            r = min(H, W) // 3
            ang = I["light_direction_deg"]
            ex, ey = int(cx - r * np.cos(np.radians(ang))), int(cy + r * np.sin(np.radians(ang)))
            cv2.arrowedLine(comp_bgr, (cx, cy), (ex, ey), (255, 255, 0), 3, tipLength=0.1)
        # Convert back to float RGB for return consistency
        comp = comp_bgr[:, :, ::-1].astype(np.float32) / 255.0
    else:
        comp = base * 0.92 + tex_layer + de_layer + sal_layer + line_layer


    # Blend all overlay layers
    comp = base * 0.92 + tex_layer + de_layer + sal_layer + line_layer
    comp = np.clip(comp, 0, 1)
    comp_bgr = (comp[:, :, ::-1] * 255).astype(np.uint8).copy()

    # 5. Subject contour (magenta edge)
    mask = S["subject_mask"]
    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if cnts and w_sal > 0.05:
        cv2.drawContours(comp_bgr, cnts, -1, (255, 0, 255), 2)

    # 6. Focal rings
    fx, fy = S["focal"]
    for r in range(6, 30, 6):
        cv2.circle(comp_bgr, (int(fx), int(fy)), r, (0, 0, 255), 1)

    # 7. Vanishing point & light arrow
    vp = P.get("vanishing_point")
    if vp:
        cv2.circle(comp_bgr, tuple(map(int, vp)), 6, (0, 255, 255), -1)
    if I["light_direction_deg"] is not None and w_light > 0.05:
        Hc, Wc = comp_bgr.shape[:2]
        cx, cy = Wc - 60, 60
        r = min(Hc, Wc) // 3
        ang = I["light_direction_deg"]
        ex, ey = int(cx - r * np.cos(np.radians(ang))), int(cy + r * np.sin(np.radians(ang)))
        cv2.arrowedLine(comp_bgr, (cx, cy), (ex, ey), (0, 150, 255), 3, tipLength=0.1)

    # Return final RGB overlay + visibility strengths
    return comp_bgr[:, :, ::-1], dict(
        w_tex=w_tex, w_de=w_de, w_sal=w_sal, w_lines=w_lines, w_light=w_light
    )
### Curatorial Paragraph Helper (Encouraging tone)
# ==========================================================
def curatorial_paragraph(M):
    C, T, P, I = M["color"], M["texture"], M["composition"], M["illumination"]

    # Color mood
    tone = {
        "analogous": "a harmonious range of kindred hues",
        "complementary": "a dynamic play of opposing colors"
    }.get(C["harmony"], "a balanced and varied palette")

    if C["mean_deltaE"] > 25:
        contrast = "vivid contrast that energizes the surface"
    elif C["mean_deltaE"] > 12:
        contrast = "moderate contrast that lends clarity"
    else:
        contrast = "gentle gradations that invite reflection"

    # Texture
    texture_phrase = (
        f"Texture carries a directional rhythm near {T['gabor_dom_angle']:.0f}°, suggesting flow and motion."
        if T["gabor_anisotropy"] > 0.5
        else "Texture remains even and tranquil, grounding the image."
    )

    # Composition
    if P["golden_delta"] < 0.12:
        comp_phrase = "The structure recalls the golden proportion, bringing calm balance."
    elif P["thirds_delta"] < 0.12:
        comp_phrase = "The focal point aligns near a rule-of-thirds intersection, evoking natural stability."
    else:
        comp_phrase = "The composition feels exploratory and open-ended."

    sym_phrase = (
        "Strong symmetry steadies the eye."
        if P["symmetry"] > 0.8 else
        "Partial symmetry offers tension and interest."
        if P["symmetry"] > 0.5 else
        "Asymmetry keeps the image vibrant and alive."
    )

    # Illumination
    light_phrase = (
        f"Light enters from about {int(I['light_direction_deg'])}°, shaping gentle tonal transitions."
        if I["light_direction_deg"] is not None
        else "Light spreads softly across the surface, creating atmospheric calm."
    )

    mood = (
        "bright and open" if I["lightness_skew"] > 0.5 else
        "quietly introspective" if I["lightness_skew"] < -0.5 else
        "balanced and contemplative"
    )

    paragraph = (
        f"This painting unfolds with {tone}, featuring {contrast}. "
        f"{texture_phrase} {comp_phrase} {sym_phrase} "
        f"{light_phrase} The overall mood feels {mood}, "
        f"encouraging a steady, unhurried gaze."
    )

    return paragraph
### Fixed Black Analysis Panel (Legend + Summary)
# ==========================================================
def right_panel_black(H, C, T, P, I, strengths, paragraph):
    """Generates the fixed black side panel with legend, key metrics, and curatorial paragraph."""
    PANEL_W = 400
    panel = np.zeros((H, PANEL_W, 3), np.uint8)
    f = cv2.FONT_HERSHEY_SIMPLEX
    x, y, lh = 16, 28, 22

    # Title
    cv2.putText(panel, "Analysis", (x, y), f, 0.8, (255, 255, 255), 2)
    y += lh + 6

    # Legend
    cv2.putText(panel, "Legend", (x, y), f, 0.7, (255, 255, 255), 1)
    y += lh
    swatches = [
        ((x, y), (x+18, y+18), (0, 0, 255), "Focal"),
        ((x, y+22), (x+18, y+40), (255, 0, 255), "Subject"),
        ((x, y+44), (x+18, y+62), (0, 255, 255), "Lines"),
        ((x, y+66), (x+18, y+84), (255, 128, 0), "Texture"),
        ((x, y+88), (x+18, y+106), (255, 0, 255), "ΔE"),
        ((x, y+110), (x+18, y+128), (255, 255, 255), "Thirds"),
        ((x, y+132), (x+18, y+150), (0, 255, 255), "Golden"),
        ((x, y+154), (x+18, y+172), (0, 150, 255), "Light→"),
        ((x, y+176), (x+18, y+194), (0, 255, 255), "Vanishing"),
    ]
    for (p1, p2, col, label) in swatches:
        cv2.rectangle(panel, p1, p2, col, -1)
        cv2.putText(panel, label, (p2[0]+8, p2[1]-2), f, 0.55, (220, 220, 220), 1)
    y += 196 + 16

    # Caption helper
    def note(strength, main):
        return main if strength >= 0.08 else f"{main} (weak / omitted visually)"

    captions = [
        note(strengths['w_de'], f"Color contrast: ΔE {C['mean_deltaE']:.1f}, {C['harmony']} harmony."),
        note(strengths['w_tex'], f"Texture anisotropy {T['gabor_anisotropy']:.2f}, dir {T['gabor_dom_angle']:.0f}°."),
        note(strengths['w_sal'], "Subject zone: central saliency focus."),
        note(strengths['w_lines'], f"Composition lines: {len(P['lines'])}, thirdsΔ {P['thirds_delta']:.3f}."),
        note(strengths['w_light'], f"Light: {I['light_direction_deg']}°, skew {I['lightness_skew']:.2f}.")
    ]

    for cap in captions:
        for line in textwrap.wrap(cap, width=42):
            if y + lh + 10 >= H:
                break
            cv2.putText(panel, line, (x, y), f, 0.55, (200, 200, 200), 1)
            y += lh
        y += 6
        if y >= H - 160:
            break

    # Add curatorial paragraph
    para_lines = textwrap.wrap(paragraph, width=44)
    y_para = min(y + 10, H - (len(para_lines) + 1) * lh - 8)
    cv2.putText(panel, "Curatorial Summary", (x, y_para), f, 0.62, (255, 255, 255), 1)
    y_para += lh
    for pl in para_lines:
        if y_para >= H - 6:
            break
        cv2.putText(panel, pl, (x, y_para), f, 0.55, (230, 230, 230), 1)
        y_para += lh

    return panel



### BLOCK 8/8 — Output Generation & Main Pipeline
# ==========================================================
# ==========================================================
# ROI Extraction + Local Curatorial Narratives
# ==========================================================

# ==========================================================
# ROI Extraction + Curatorial Narratives (context-rich)
# ==========================================================

def extract_regions_of_interest(interest_map, n_max=5):
    """
    Detect spatially distinct regions of high interest using weighted clustering.
    Ensures regions are well-separated and contextually balanced.
    """
    H, W = interest_map.shape
    im = np.clip(interest_map, 0, 1)
    flat = im.flatten()
    idx = np.argsort(flat)[::-1]  # sort by descending interest

    # sample top 8% interest points for clustering
    cutoff = int(len(flat) * 0.08)
    ys, xs = np.divmod(idx[:cutoff], W)
    pts = np.column_stack([xs, ys]).astype(np.float32)
    if len(pts) < 10:
        return []

    # K-means to find region centroids
    k = min(n_max, 1 + len(pts)//1500)
    _, labels, centers = cv2.kmeans(pts, k, None,
                                    (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 40, 1.0),
                                    8, cv2.KMEANS_PP_CENTERS)
    rois = []
    for (cx, cy) in centers:
        cx, cy = int(cx), int(cy)
        radius = int(min(H, W) * 0.12)
        x1, y1, x2, y2 = max(0, cx-radius), max(0, cy-radius), min(W, cx+radius), min(H, cy+radius)
        score = float(np.mean(im[y1:y2, x1:x2]))
        rois.append(dict(bbox=(x1, y1, x2-x1, y2-y1), center=(cx, cy), score=score))

    # sort by local interest and keep unique spatial regions
    rois = sorted(rois, key=lambda r: r["score"], reverse=True)[:n_max]
    used = []
    distinct = []
    for r in rois:
        cx, cy = r["center"]
        if all(np.hypot(cx - u[0], cy - u[1]) > min(H, W) * 0.18 for u in used):
            distinct.append(r)
            used.append((cx, cy))
    return distinct
def draw_roi_overlays(img, rois):
    """
    Overlay shaded color regions of interest on the artwork.
    Each ROI gets a unique translucent color with a clear label.
    """
    colors = [
        (255, 80, 80),
        (255, 200, 50),
        (80, 255, 100),
        (80, 200, 255),
        (255, 120, 220)
    ]
    overlay = img.copy().astype(np.float32)
    alpha = 0.35

    for i, r in enumerate(rois):
        x, y, w, h = r["bbox"]
        color = colors[i % len(colors)]
        roi_mask = np.zeros(img.shape[:2], np.uint8)
        cv2.rectangle(roi_mask, (x, y), (x + w, y + h), 255, -1)
        roi_color = np.full_like(img, color, np.uint8)
        overlay = np.where(
            roi_mask[:, :, None] > 0,
            (1 - alpha) * overlay + alpha * roi_color,
            overlay
        )

        label = f"ROI {i + 1}"
        cv2.putText(
            overlay, label, (x + 6, y + 22),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2, cv2.LINE_AA
        )
        cv2.putText(
            overlay, label, (x + 6, y + 22),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA
        )

    return np.clip(overlay, 0, 255).astype(np.uint8)


def describe_regions(rois, C, T, S, P, I):
    """
    Generate natural, human-style curatorial paragraphs for each ROI.
    Each summary references unique qualities of that area.
    """
    texts = []
    H, W = S["saliency_map"].shape
    fx, fy = S["focal"]
    light_dir = I["light_direction_deg"]

    for i, r in enumerate(rois):
        x, y, w, h = r["bbox"]
        cx, cy = x + w / 2, y + h / 2
        s = r["score"]

        # Normalize and measure relative factors
        region_sal = np.mean(S["saliency_map"][y:y+h, x:x+w]) / 255.0
        region_tex = np.mean(T["gabor_map"][y:y+h, x:x+w])
        region_de  = np.mean(C["contrast_map"][y:y+h, x:x+w])
        region_line = np.mean(_line_density_map(H, W, P["lines"])[y:y+h, x:x+w])
        region_light = abs(I["lightness_skew"])

        # Determine dominant traits
        mC = norm01(region_de, 0.05, 0.25)
        mT = norm01(region_tex, 0.1, 0.9)
        mS = region_sal
        mL = norm01(region_line, 0.05, 0.35)
        mI = norm01(region_light, 0.05, 0.6)

        dominant = []
        if mC > 0.45: dominant.append("color contrast")
        if mT > 0.45: dominant.append("texture")
        if mS > 0.45: dominant.append("saliency")
        if mL > 0.45: dominant.append("compositional lines")
        if mI > 0.45: dominant.append("illumination")
        dom_str = ", ".join(dominant) if dominant else "a subtle blend of qualities"

        # Position-based phrasing
        tone = "upper" if cy < H / 3 else "central" if H / 3 <= cy <= 2 * H / 3 else "lower"
        lateral = "left" if cx < W / 3 else "right" if cx > 2 * W / 3 else "center"
        structure_tone = "structured and intentional" if mL > 0.6 else "gentle and organic"

        summary = (
            f"Region {i + 1}, located in the {tone}-{lateral} area, stands out through {dom_str}. "
            f"The balance here feels {structure_tone}, "
            f"encouraging a closer, more reflective look into this part of the painting."
        )
        texts.append(summary)

    return texts


def analyze_artwork_v17(path, verbose=True, thorough=True):
    """Full dynamic analysis with individual visual outputs and clear progress updates."""
    start = time.time()
    print(f"🖼️ Loading image: {path}")
    img = load_rgb(path)
    H0, W0 = img.shape[:2]
    print(f"   • Image size: {W0}×{H0}")

    # Rescale for fidelity
    if thorough:
        img, _ = resize_min_pixels(img, target_pixels=1_600_000)
        H, W = img.shape[:2]
        if (H, W) != (H0, W0):
            print(f"   • Rescaled for fidelity: {W}×{H}")
    else:
        H, W = H0, W0

    # Directory prep
    out_dir = os.path.splitext(path)[0] + "_analysis"
    ensure_dir(out_dir)

    # ---- Analyses ----
    print("🎨 Running color analysis…", end=" "); C = color_analysis(img); print("✓")
    print("🪶 Running texture analysis…", end=" "); T = gabor_pyramid(cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)); print("✓")
    print("🔥 Running saliency analysis…", end=" "); S = saliency_analysis(img); print("✓")
    print("📐 Running composition analysis…", end=" "); P = composition_analysis(img, S['focal']); print("✓")
    print("💡 Running illumination analysis…", end=" "); I = illumination_analysis(img); print("✓")
        # ---- ROI Interest Map (debug) ----
    print("🌐 Building ROI interest map…", end=" ")
    interest = roi_interest_map(C, T, S, P, I, shape=img.shape[:2])
    print("✓")

    # ---- Save ROI heatmap preview ----
    heat_dbg = roi_interest_heatmap_overlay(img, interest, alpha=0.35)
    cv2.imwrite(os.path.join(out_dir, "roi_interest_heatmap.png"),
                cv2.cvtColor(heat_dbg, cv2.COLOR_RGB2BGR))
    # ---- Extract and describe ROIs ----
    print("🔍 Extracting regions of interest…", end=" ")
    rois = extract_regions_of_interest(interest)
    roi_overlay = draw_roi_overlays(img, rois)
    roi_texts = describe_regions(rois, C, T, S, P, I)
    cv2.imwrite(os.path.join(out_dir, "roi_regions_overlay.png"),
                cv2.cvtColor(roi_overlay, cv2.COLOR_RGB2BGR))
    print(f"{len(rois)} regions ✓")

    # Save regional curatorial summaries
    with open(os.path.join(out_dir, "roi_summaries.txt"), "w", encoding="utf-8") as f:
        for t in roi_texts:
            f.write(t + "\n\n")


    # ---- Generate overlays ----
    print("✨ Building visual overlays…", end=" ")
    art_overlay, strengths = overlays_on_artwork(img, C, T, S, P, I)
    print("✓")

    # ---- Curatorial summary ----
    print("🖋️ Writing curatorial summary…", end=" ")
    paragraph = curatorial_paragraph(dict(color=C, texture=T, composition=P, illumination=I))
    print("✓")

    # ---- Generate right panel ----
    panel = right_panel_black(art_overlay.shape[0], C, T, P, I, strengths, paragraph)
    final = np.hstack([art_overlay, panel])
    final = safe_gamma(final)

    # ---- Save outputs ----
    print("💾 Saving images…", end=" ")
    out_final = os.path.join(out_dir, "composite_full.png")
    cv2.imwrite(out_final, final)

    # Individual analysis images
    indiv = [
        ("color_overlay.png", C, "Color Analysis", "Shows chromatic clusters and ΔE contrast regions."),
        ("texture_overlay.png", T, "Texture Analysis", "Highlights directional energy and surface anisotropy."),
        ("saliency_overlay.png", S, "Saliency Map", "Visualizes attention-weighted subject areas."),
        ("composition_overlay.png", P, "Composition Analysis", "Shows detected structure and balance."),
        ("illumination_overlay.png", I, "Illumination Map", "Indicates light direction and tonal gradient.")
    ]
    for fname, data, title, desc in indiv:
        mode = fname.split("_")[0]  # e.g. "color", "texture", ...
        overlay, _ = overlays_on_artwork(img, C, T, S, P, I, mode=mode)
        overlay = safe_gamma(overlay)
        cv2.imwrite(os.path.join(out_dir, fname), overlay)
    print("✓")

    # ---- Save summary and metrics ----
    print("📄 Saving textual outputs…", end=" ")
    with open(os.path.join(out_dir, "curatorial_summary.txt"), "w", encoding="utf-8") as f:
        f.write(paragraph + "\n")
    with open(os.path.join(out_dir, "metrics.json"), "w", encoding="utf-8") as f:
        json.dump(json_safe(dict(color=C, texture=T, saliency=S, composition=P, illumination=I, strengths=strengths)), f, indent=2)
    print("✓")

    # ---- Wrap up ----
    elapsed = time.time() - start
    print(f"\n✅ Complete! Total runtime: {elapsed:.2f}s")
    print(f"📁 Outputs saved in: {out_dir}")

# -----------------------------
# Run locally
# -----------------------------
if __name__ == "__main__":
    IMAGE_PATH = "/Users/alievanayasso/Documents/SlowMA/point.jpg"
    analyze_artwork_v17(IMAGE_PATH, verbose=True, thorough=True)


🖼️ Loading image: /Users/alievanayasso/Documents/SlowMA/point.jpg
   • Image size: 750×600
   • Rescaled for fidelity: 1414×1131
🎨 Running color analysis… ✓
🪶 Running texture analysis… ✓
🔥 Running saliency analysis… ✓
📐 Running composition analysis… ✓
💡 Running illumination analysis… ✓
🌐 Building ROI interest map… ✓
🔍 Extracting regions of interest… 5 regions ✓
✨ Building visual overlays… ✓
🖋️ Writing curatorial summary… ✓
💾 Saving images… ✓
📄 Saving textual outputs… ✓

✅ Complete! Total runtime: 13.34s
📁 Outputs saved in: /Users/alievanayasso/Documents/SlowMA/point_analysis
